## Imports and Functions

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)

colors = pc.qualitative.D3
angstrom = "\u212B"
theta = "\u03B8"
phi = "\u03A6"

In [ ]:
from igorwriter import IgorWave


def hdf5_to_ibw(ddir, hdf5_filename, ibw_filename, overwrite=False):
    """Loads .h5 file, converts to .ibw using IgorWave."""

    hdf5_filepath = os.path.join(ddir, hdf5_filename)
    ibw_filepath = os.path.join(ddir, ibw_filename)

    if not overwrite and os.path.exists(ibw_filepath):
        raise FileExistsError(
            f"{ibw_filepath} already exists and overwrite is set to False."
        )

    with h5py.File(hdf5_filepath, "r") as f:
        data = f["data"][()]
        axes = []
        axes_names = [
            "theta",
            "energy",
            "phi",
        ]  # Change these to match your axes labels
        for name in axes_names:
            axis = f[name][()]
            axes.append(axis)

    axis_start = [axis[0] for axis in axes]
    axis_delta = [np.diff(axis[:2])[0] if axis.size > 1 else 0 for axis in axes]

    wave = IgorWave(data, name=ibw_filename.split(".")[0])

    for axis, start, delta, label in zip(
        ["x", "y", "z"], axis_start, axis_delta, axes_names
    ):
        wave.set_dimscale(axis, start, delta, units=label)

    wave.save(ibw_filepath, image=False)

    return ibw_filepath

In [ ]:
def generate_FFT_filtered_dataset(
    theta, energy, phi, data, fp, fn, int_range=0.0, overwrite=False, new_fn=None
):
    new_fn = (
        new_fn
        if new_fn
        else f"{os.path.splitext(fn)[0]}_filteredFFT_{int_range:.2f}int.h5"
    )
    new_fn = os.path.join(fp, new_fn)
    if not overwrite and os.path.exists(new_fn):
        raise FileExistsError(f"{new_fn} already exists")
    new_data = []
    for p in phi:
        val = p
        xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(
            x=theta,
            y=energy,
            z=phi,
            data=data,
            slice_dim="z",
            slice_val=val,
            int_range=int_range,
        )
        fft_data = filter_functions.fft2d_mask(dataslice, plot=False)
        new_data.append(fft_data)
    new_data = np.array(new_data).T
    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data
        axes_names = [
            "theta",
            "energy",
            "phi",
        ]  # Change these to match your axes labels
        axes = [theta, energy, phi]
        for axis, name in zip(axes, axes_names):
            f[name] = axis
    return new_fn

In [ ]:
def generate_streak_corrected_dataset(
    ad,
    fp,
    fn,
    correction_array,
    xlim=None,
    ylim=None,
    x_bin=1,
    y_bin=1,
    int_range=0.0,
    overwrite=False,
    new_fn=None,
):
    new_fn = (
        new_fn
        if new_fn
        else f"{os.path.splitext(fn)[0]}_streakCorrect_{int_range:.2f}int.h5"
    )
    new_fn = os.path.join(fp, new_fn)
    if not overwrite and os.path.exists(new_fn):
        raise FileExistsError(f"{new_fn} already exists")
    new_data = []
    for p in ad.phi_or_time:
        val = p
        xaxis, yaxis, dataslice = tr_functions.slice_datacube(
            ad,
            slice_dim="z",
            slice_val=val,
            int_range=int_range,
            xlim=xlim,
            ylim=ylim,
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
        corrected_data = correction_array * dataslice
        new_data.append(corrected_data)
    new_data = np.array(new_data).T
    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data
        axes_names = [
            "theta",
            "energy",
            "phi",
        ]  # Change these to match your axes labels
        axes = [xaxis, yaxis, ad.phi_or_time]
        for axis, name in zip(axes, axes_names):
            f[name] = axis
    return new_fn

In [ ]:
def average_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.mean(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

# 5-6 ML Film (XUV): HOMO

## Original raw averaged data

In [ ]:
# ## Convert .ibw to .h5 ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"

# # Convert ibw to hdf5
# fn = r"FS2andFS4_avg.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, phi, energy = HDF5_loader.load_hdf5(
#     ddir, r"FS2andFS4_avg.h5"
# )  # load data from hdf5
# data.shape, theta.shape, phi.shape, energy.shape

In [ ]:
# ## FFT filter dataset ##

# # Load original dataset
# fp = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# fn = r"FS2andFS4_avg.h5"
# data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

# # Generate FFT filtered dataset
# generate_FFT_filtered_dataset(
#     theta=theta, energy=energy, phi=phi, data=data, fp=fp, fn=fn
# )

## Average FS2 and FS4 again (to get rid of extra dataset error)

In [ ]:
# ## Load multiple datasets and average ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\FS2"

# files = [
#     "FS2_000_1.h5",
#     "FS2_001_2.h5",
#     "FS2_002_3.h5",
#     "FS2_003_4.h5",
#     "FS2_004_5.h5",
#     "FS2_005_6.h5",
#     "FS2_006_7.h5",
# ]

# ## Old Way:
# ## Load single dataset for axes info
# # fn = r"FS2_000_1.h5"
# # data, ss, cs, p = HDF5_loader.data_from_hdf_2022(fp, fn)
# # data_avg = HDF5_loader.avg_array_from_hdfs(fp, fns)  # average data
# # datat = np.swapaxes(data_avg, 2, 0)
# ## Export averaged data to hdf5 file
# # HDF5_loader.avg_data_hdf(
# #     fp, "FS2_ali_avg", datat, p=p, slice_scale=ss, channel_scale=cs
# # )

# ## New Way:
# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

# average_timescans(files, ddir, "FS2_Ali_avg.h5")

In [ ]:
# ## Load multiple datasets and average ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\FS4"

# files = [
#     "FS4_001_1.h5",
#     "FS4_002_2.h5",
#     "FS4_003_3.h5",
#     "FS4_004_4.h5",
# ]

# ## Old Way:
# ## Load single dataset for axes info
# # fn = r"FS4_001_1.h5"
# # data, ss, cs, p = HDF5_loader.data_from_hdf_2022(fp, fn)
# # data_avg = HDF5_loader.avg_array_from_hdfs(fp, fns)  # average data
# # datat = np.swapaxes(data_avg, 2, 0)
# ## Export averaged data to hdf5 file
# # HDF5_loader.avg_data_hdf(
# #     fp, "FS4_ali_avg", datat, p=p, slice_scale=ss, channel_scale=cs
# # )

# ## New Way:
# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

# average_timescans(files, ddir, "FS4_Ali_avg.h5")

In [ ]:
# ## Average FS2_ali_avg and FS4_ali_avg ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"

# files = [
#     "FS2_ali_avg.h5",
#     "FS4_ali_avg.h5",
# ]

# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

# average_timescans(files, ddir, "FS2andFS4_Ali_avg.h5")

In [ ]:
# ## FFT filter dataset ##

# # Load original dataset
# fp = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# fn = r"FS2andFS4_ali_avg.h5"
# data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

# # Generate FFT filtered dataset
# generate_FFT_filtered_dataset(
#     theta=theta, energy=energy, phi=phi, data=data, fp=fp, fn=fn
# )

## Load Data

In [ ]:
## Load averaged, FFT filtered ##

ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# files = ["FS2andFS4_avg_filteredFFT_0.00int.h5"]  # improperly averaged
files = ["FS2andFS4_ali_avg_filteredFFT_0.00int.h5"]  # properly averaged

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
ad = ARPES_DATA[files[0]]
for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad, k).shape}")
print(f"Data.shape = {ad.data.shape}")

## Analysis

In [ ]:
## Initial params ##
slice_dim = "z"
EF = 18.48

slice_val = 0
int_range = 100

x_bin = 1
y_bin = 1

# xlim = None
xlim = (-21, 16.3)
ylim = None

In [ ]:
theta = "\u03B8"
phi = "\u03A6"

# title=f"C<sub>60</sub> Band Structure (k<sub>x</sub> = {slice_val})"
# xaxis_title=f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]"
# yaxis_title=f"E - E<sub>F</sub> [eV]"

title = f"C<sub>60</sub> Band Structure (All {phi})"
xaxis_title = f"{theta}"
yaxis_title = f"E<sub>K</sub> [eV]"

In [ ]:
## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=False,
    plot_data=False,
)

# Plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)

fig.update_coloraxes(cmin=0, cmax=0.2)

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"MDC of HOMO",
    xaxis_title=xaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
)

# y_1d, col = tr_functions.get_1d_x_slice(
#     x=x_2d, y=y_2d, data=d_2d, ylims=(15.8, 17.1), x_range=None
# )

x_1d, row = tr_functions.get_1d_y_slice(
    x=x_2d, y=y_2d, data=d_2d, xlims=None, y_range=None
)

# Plot Data
fig.add_trace(
    go.Scatter(
        x=x_1d,
        y=analysis_functions.norm_data(row),
        name="data",
        line=dict(color=colors[0]),
    )
)

fig.show()

In [ ]:
# Plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)

fig.update_coloraxes(cmin=0, cmax=0.2)

fig.add_trace(
    go.Scatter(
        x=x_1d,
        y=analysis_functions.norm_data(row) + 16,
        name="MDC",
        line=dict(color=colors[1]),
    )
)

fig.update_layout(
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=0.9, xanchor="right", x=0.98),
)

fig.show()

In [ ]:
## Fit Data ##

x = x_1d
data = row

offset_type = "constant"

## Offset
# c = np.mean(data)
c = 0
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## 4th order polynomial
poly = lm.models.PolynomialModel(degree=4)

full_model = poly + offset

fit = full_model.fit(data, x=x)

fit.plot()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"MDC of HOMO",
    xaxis_title=xaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
)

x_1d, row = tr_functions.get_1d_y_slice(
    x=x_2d, y=y_2d, data=d_2d, xlims=None, y_range=None
)

# Plot Data
fig.add_trace(
    go.Scatter(
        x=x_1d,
        y=row,
        name="data",
        line=dict(color=colors[0]),
    )
)

fig.add_trace(go.Scatter(x=x_1d, y=fit.eval(x=x_1d), name="fit"))

fig.show()

In [ ]:
## Generate Correction Ratio Array ##

correction_ratio = fit.eval(x=x_1d) / row
print(correction_ratio.shape, d_2d.shape)  # Output: (753,) (1064, 753)

repeated_correction = np.tile(correction_ratio, (1064, 1))
print(repeated_correction.shape)  # Output: (1064, 753)

corrected_d_2d = repeated_correction * d_2d

In [ ]:
## Check Equivalence in Array ##

np.sum(
    repeated_correction[0, :] == repeated_correction[1, :]
)  # each of the 1064 rows is the same

In [ ]:
# Plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d,
        y=y_2d,
        z=analysis_functions.norm_data(corrected_d_2d),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=0.2)

fig.show()

## Generate Streak-Corrected Dataset

In [ ]:
# fp = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# # fn = r"FS2andFS4_avg_filteredFFT_0.00int.h5"  # improperly averaged
# fn = r"FS2andFS4_ali_avg_filteredFFT_0.00int.h5"  # properly averaged

# xlim = (-21, 16.3)
# ylim = None
# generate_streak_corrected_dataset(
#     ad, fp, fn, repeated_correction, xlim=xlim, ylim=ylim, overwrite=True
# )

## Load Streak-Corrected Data

In [ ]:
## Load averaged, FFT filtered, streak corrected data ##

ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# files = ["FS2andFS4_avg_filteredFFT_0.00int_streakCorrect_0.00int.h5"]  # improperly averaged
files = [
    "FS2andFS4_ali_avg_filteredFFT_0.00int_streakCorrect_0.00int.h5"
]  # properly averaged

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
ad_corrected = ARPES_DATA[files[0]]
for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad_corrected, k).shape}")
print(f"Data.shape = {ad_corrected.data.shape}")

## Analysis

In [ ]:
## Initial params ##
slice_dim = "z"
EF = 18.48

slice_val = 0
int_range = 0.1

x_bin = 1
y_bin = 1

xlim = None
# xlim = (-21, 16.3)
ylim = None

In [ ]:
theta = "\u03B8"
phi = "\u03A6"

title = f"C<sub>60</sub> Band Structure ({phi} = {slice_val})"
xaxis_title = f"{theta}"
yaxis_title = f"E<sub>K</sub> [eV]"

In [ ]:
## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_corrected,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=False,
    plot_data=False,
)

# Plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)

fig.update_coloraxes(cmin=0, cmax=0.2)

fig.show()

## FS Reconstruction

In [ ]:
## Initial params ##
slice_dim = "y"
EF = 18.48

slice_val = 16.8
# slice_val = 16.4
# slice_val = 16.0

int_range = 0.1

x_bin = 1
y_bin = 1

In [ ]:
# title = f"C<sub>60</sub> HOMO ({slice_val} eV)"
# yaxis_title = f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]"
# xaxis_title = f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]"

title = f"C<sub>60</sub> HOMO ({slice_val} eV)"
yaxis_title = f"{phi}"
xaxis_title = f"{theta}"

In [ ]:
xlim = (-15, 15)
ylim = None

## Get original data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    dtick_y=10,
    height=600,
    width=500,
)

## Plot Data
fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)


# hexagon = polygons.gen_polygon(6, 0.42, translation=(0, -0.375 * 2), rotation=30)
# fig = polygons.plot_polygon(
#     hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
# )

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

In [ ]:
## Limit Dataset ##

## k-corrected
# xlim = (-0.52, 0.52)
# ylim = (-1.25, 0.15)

## angle
xlim = (-15, 15)
ylim = None

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_corrected,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    dtick_y=10,
    height=600,
    width=500,
)

## Plot Data
fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)


# hexagon = polygons.gen_polygon(6, 0.42, translation=(0, -0.375 * 2), rotation=30)
# fig = polygons.plot_polygon(
#     hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
# )

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

## Export FFT, Streak-Corrected Dataset to .ibw for k-correction

In [ ]:
# ## Convert hdf5 to ibw ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# hdf5_file = "FS2andFS4_ali_avg_filteredFFT_0.00int_streakCorrect_0.00int.h5"
# ibw_file = "FS2andFS4_ali_avg_FFT_streakCorrect.ibw"

# hdf5_to_ibw(ddir, hdf5_file, ibw_file, overwrite=True)

In [ ]:
# ## Check this worked by converting ibw to hdf5 ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# fn = "FS2andFS4_ali_avg_FFT_streakCorrect.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

In [ ]:
# files = ["FS2andFS4_ali_avg_FFT_streakCorrect.h5"]

# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# ad_export = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad_export, k).shape}")
# print(f"Data.shape = {ad_export.data.shape}")

# 5-6 ML Film (XUV): HOMO-1

In [ ]:
# ## Convert .ibw to .h5 ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"

# # Convert ibw to hdf5
# fn = r"FS3_avg.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, phi, energy = HDF5_loader.load_hdf5(
#     ddir, r"FS3_avg.h5"
# )  # load data from hdf5
# data.shape, theta.shape, phi.shape, energy.shape

In [ ]:
# ## FFT filter dataset ##

# # Load original dataset
# fp = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# fn = r"FS3_avg.h5"
# data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

# # Generate FFT filtered dataset
# generate_FFT_filtered_dataset(
#     theta=theta, energy=energy, phi=phi, data=data, fp=fp, fn=fn
# )

## Load Data

In [ ]:
## 5-6 ML Film (XUV): HOMO-1 ##

ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
files = ["FS3_avg_filteredFFT_0.00int.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_homo1 = ARPES_DATA[files[0]]

## Analysis

In [ ]:
## Initial params ##
slice_dim = "z"
EF = 18.48

slice_val = 0
int_range = 100

x_bin = 1
y_bin = 1

# xlim = None
xlim = (-21, 16.3)
ylim = None

In [ ]:
# title=f"C<sub>60</sub> Band Structure (k<sub>x</sub> = {slice_val})"
# xaxis_title=f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]"
# yaxis_title=f"E - E<sub>F</sub> [eV]"

title = f"C<sub>60</sub> Band Structure (All {phi})"
xaxis_title = f"{theta}"
yaxis_title = f"E<sub>K</sub> [eV]"

In [ ]:
## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_homo1,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=False,
    plot_data=False,
)

# Plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)

fig.update_coloraxes(cmin=0, cmax=0.2)

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"MDC of HOMO-1",
    xaxis_title=xaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
)

# y_1d, col = tr_functions.get_1d_x_slice(
#     x=x_2d, y=y_2d, data=d_2d, ylims=(15.8, 17.1), x_range=None
# )

x_1d, row = tr_functions.get_1d_y_slice(
    x=x_2d, y=y_2d, data=d_2d, xlims=None, y_range=None
)

# Plot Data
fig.add_trace(
    go.Scatter(
        x=x_1d,
        y=analysis_functions.norm_data(row),
        name="data",
        line=dict(color=colors[0]),
    )
)

fig.show()

In [ ]:
## Fit Data ##

x = x_1d
data = row

offset_type = "constant"

## Offset
# c = np.mean(data)
c = 0
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## 4th order polynomial
poly = lm.models.PolynomialModel(degree=4)

full_model = poly + offset

fit = full_model.fit(data, x=x)

fit.plot()

In [ ]:
## Generate Correction Ratio Array ##

correction_ratio = fit.eval(x=x_1d) / row
print(correction_ratio.shape, d_2d.shape)  # Output: (753,) (1064, 753)

repeated_correction = np.tile(correction_ratio, (1064, 1))
print(repeated_correction.shape)  # Output: (1064, 753)

corrected_d_2d = repeated_correction * d_2d

In [ ]:
## Check Equivalence in Array ##

np.sum(
    repeated_correction[0, :] == repeated_correction[1, :]
)  # each of the 1064 rows is the same

In [ ]:
# Plot data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d,
        y=y_2d,
        z=analysis_functions.norm_data(corrected_d_2d),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=0.2)

fig.show()

## Generate Streak-Corrected Dataset

In [ ]:
# ## Generate Streak Corrected Dataset ##

# fp = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# fn = r"FS3_avg_filteredFFT_0.00int.h5"

# xlim = (-21, 16.3)
# ylim = None
# generate_streak_corrected_dataset(
#     ad_homo1, fp, fn, repeated_correction, xlim=xlim, ylim=ylim, overwrite=True
# )

## Load Streak-Corrected Data

In [ ]:
## 5-6 ML Film (XUV): HOMO-1 ##

ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
files = ["FS3_avg_filteredFFT_0.00int_streakCorrect_0.00int.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_homo1_corrected = ARPES_DATA[files[0]]

## Analysis

In [ ]:
## Initial params ##
slice_dim = "y"
EF = 18.48

slice_val = 15.6
# slice_val = 15.2
# slice_val = 14.7

int_range = 0.1

x_bin = 1
y_bin = 1

In [ ]:
theta = "\u03B8"
phi = "\u03A6"

# title = f"C<sub>60</sub> HOMO ({slice_val} eV)"
# yaxis_title = f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]"
# xaxis_title = f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]"

title = f"C<sub>60</sub> HOMO ({slice_val} eV)"
yaxis_title = f"{phi}"
xaxis_title = f"{theta}"

In [ ]:
xlim = (-15, 15)
ylim = None

## Get original data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_homo1,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    dtick_y=10,
    height=600,
    width=500,
)

## Plot Data
fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)


# hexagon = polygons.gen_polygon(6, 0.42, translation=(0, -0.375 * 2), rotation=30)
# fig = polygons.plot_polygon(
#     hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
# )

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

In [ ]:
## Limit Dataset ##

## k-corrected
# xlim = (-0.52, 0.52)
# ylim = (-1.25, 0.15)

## angle
xlim = (-15, 15)
ylim = None

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_homo1_corrected,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    dtick_y=10,
    height=600,
    width=500,
)

## Plot Data
fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)


# hexagon = polygons.gen_polygon(6, 0.42, translation=(0, -0.375 * 2), rotation=30)
# fig = polygons.plot_polygon(
#     hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
# )

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

## Export FFT, Streak-Corrected Dataset to .ibw for k-correction

In [ ]:
# ## Convert hdf5 to ibw ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"
# hdf5_file = "FS3_avg_filteredFFT_0.00int_streakCorrect_0.00int.h5"
# ibw_file = "FS3_avg_FFT_streakCorrect.ibw"

# hdf5_to_ibw(ddir, hdf5_file, ibw_file, overwrite=True)